In [3]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')

sys.path.append(work_dir)

In [13]:
from src.model.models import Candidates
from src.database.dbconnection import getconnection
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
from transform import DataTransform
from sqlalchemy.exc import SQLAlchemyError



In [17]:
engine = getconnection()
Session = sessionmaker(bind=engine)
session = Session()

Conected successfully to database workshop1!


In [19]:
try:
    if inspect(engine).has_table('Candidates'):
        # La tabla existe, elimínala primero
        Candidates.__table__.drop(engine)
    
    # Crea la tabla (ya sea que existiera o no)
    Candidates.__table__.create(engine)
    print("Table created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
finally:
    engine.dispose()

Table created successfully.


In [22]:
try:
    file = DataTransform('../../data/candidates.csv')
    file.insert_id()
    file.rename()
    

    # Usar el engine para manejar la conexión
    file.df.to_sql('Candidates', con=engine, if_exists='replace', index=False)
    print("Data uploaded")

except SQLAlchemyError as e:
    print(f"Database error: {e}")

except Exception as e:
    print(f"Error: {e}")

finally:
    # Cierra la conexión si es necesario, pero asegúrate de que `connection` es un objeto que tiene un método `dispose()`
    if hasattr(engine, 'dispose'):
        engine.dispose()
    # Si también estás usando `session`, ciérrala si es necesario
    if 'session' in locals():
        session.close()

Data uploaded
